In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, mean_squared_error, mean_absolute_error
import pandas as pd
import numpy as np
import tensorflow as tf
import keras_tuner as kt
import keras as kr
import matplotlib.pyplot as plt


In [2]:
biden_v_trump24 = pd.read_csv('../election_resources/biden_v_trump24.csv')
biden_v_trump24.head()

,samplesize,party,results,favorability
0,1500.0,d,0,-22.33400
1,1178.0,d,0,-20.66730
2,1076.0,d,0,-12.66730
3,2000.0,d,0,-14.43379
4,1500.0,d,0,-29.66750


In [3]:
y = biden_v_trump24['results']

In [4]:
biden_v_trump24 = biden_v_trump24.drop(columns=['results'])

In [5]:
biden_v_trump24.nunique()

samplesize      409
party             2
favorability    632
dtype: int64

In [6]:
biden_v_trump24.columns

Index(['samplesize', 'party', 'favorability'], dtype='object')

In [7]:
biden_v_trump24 = pd.get_dummies(biden_v_trump24, dtype=float)
biden_v_trump24.head()

,samplesize,favorability,party_d,party_r
0,1500.0,-22.33400,1.0,0.0
1,1178.0,-20.66730,1.0,0.0
2,1076.0,-12.66730,1.0,0.0
3,2000.0,-14.43379,1.0,0.0
4,1500.0,-29.66750,1.0,0.0


In [8]:
biden_v_trump24.shape

(1662, 4)

In [9]:
nn_model = tf.keras.models.load_model('trained_model_president.h5')
nn_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 21)                105       
                                                                 
 dense_10 (Dense)            (None, 17)                374       
                                                                 
 dense_11 (Dense)            (None, 23)                414       
                                                                 
 dense_12 (Dense)            (None, 23)                552       
                                                                 
 dense_13 (Dense)            (None, 15)                360       
                                                                 
 dense_14 (Dense)            (None, 11)                176       
                                                                 
 dense_15 (Dense)            (None, 1)                

In [10]:
scaler = StandardScaler()
X_scaler = scaler.fit(biden_v_trump24)
X_scaled = X_scaler.transform(biden_v_trump24)


In [11]:
X_scaled

array([[-0.12783023, -0.33801254,  1.00240964, -1.00240964],
       [-0.33224768, -0.22363611,  1.00240964, -1.00240964],
       [-0.39700104,  0.32535979,  1.00240964, -1.00240964],
       ...,
       [ 0.31655553, -0.49813406, -0.99759615,  0.99759615],
       [-0.12783023, -0.06350087, -0.99759615,  0.99759615],
       [ 0.31655553, -0.65826244, -0.99759615,  0.99759615]])

In [12]:
predictions = nn_model.predict(X_scaled)

52/52 [==============================] - 2s 2ms/step


In [13]:
model_loss, model_accuracy = nn_model.evaluate(predictions,y,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

ValueError: in user code:

    File "c:\Users\micha\anaconda3\envs\ml_main\lib\site-packages\keras\engine\training.py", line 1727, in test_function  *
        return step_function(self, iterator)
    File "c:\Users\micha\anaconda3\envs\ml_main\lib\site-packages\keras\engine\training.py", line 1713, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\micha\anaconda3\envs\ml_main\lib\site-packages\keras\engine\training.py", line 1701, in run_step  **
        outputs = model.test_step(data)
    File "c:\Users\micha\anaconda3\envs\ml_main\lib\site-packages\keras\engine\training.py", line 1665, in test_step
        y_pred = self(x, training=False)
    File "c:\Users\micha\anaconda3\envs\ml_main\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\micha\anaconda3\envs\ml_main\lib\site-packages\keras\engine\input_spec.py", line 277, in as

In [14]:
predictions

array([[0.00643122],
       [0.00646484],
       [0.00659455],
       ...,
       [0.95503944],
       [0.9514751 ],
       [0.945741  ]], dtype=float32)

In [40]:
predictions_df = pd.DataFrame(predictions, columns=['predictions'])

In [15]:
#auc_roc = roc_auc_score(y, predictions)
mse = mean_squared_error(y, predictions)
mae = mean_absolute_error(y, predictions)
#print(f'AUC-ROC score: {auc_roc}')
print(f'MSE score: {mse}')
print(f'MAE score: {mae}')

MSE score: 0.4197426365300657
MAE score: 0.47130228359998616


In [41]:
predictions_df['results']=results
predictions_df.head()

,predictions,results
0,0.005494,0
1,0.001647,0
2,0.002206,0
3,0.000069,0
4,0.039327,0


In [42]:
predictions_df.to_csv('predictions.csv', index=False)

In [43]:
predictions_df.head()

,predictions,results
0,0.005494,0
1,0.001647,0
2,0.002206,0
3,0.000069,0
4,0.039327,0
